In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

Using TensorFlow backend.


In [2]:
# initialize the data and labels
data = []
labels = []

[INFO] loading images...


In [3]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('animals')))
random.seed(42)
random.shuffle(imagePaths)

In [4]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, resize the image to be 32x32 pixels (ignoring
    # aspect ratio), flatten the image into 32x32x3=3072 pixel image
    # into a list, and store the image in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [5]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)

In [6]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [7]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [8]:
# define the 3072-1024-512-3 architecture using Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

In [9]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 150

# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] training network...


In [10]:
lb.classes_

array(['cats', 'dogs', 'panda'], dtype='<U5')

In [11]:
# train the neural network
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=32)

Train on 2250 samples, validate on 750 samples
Epoch 1/150
2250/2250 [==============================] - 2s 847us/step - loss: 1.1016 - accuracy: 0.3444 - val_loss: 1.0954 - val_accuracy: 0.3147
Epoch 2/150
2250/2250 [==============================] - 2s 801us/step - loss: 1.0853 - accuracy: 0.3898 - val_loss: 1.0874 - val_accuracy: 0.3160
Epoch 3/150
2250/2250 [==============================] - 2s 819us/step - loss: 1.0697 - accuracy: 0.4089 - val_loss: 1.0464 - val_accuracy: 0.4987
Epoch 4/150
2250/2250 [==============================] - 2s 802us/step - loss: 1.0496 - accuracy: 0.4596 - val_loss: 1.0724 - val_accuracy: 0.3280
Epoch 5/150
2250/2250 [==============================] - 2s 789us/step - loss: 1.0410 - accuracy: 0.4498 - val_loss: 1.0397 - val_accuracy: 0.5040
Epoch 6/150
2250/2250 [==============================] - 2s 808us/step - loss: 1.0219 - accuracy: 0.4876 - val_loss: 1.0240 - val_accuracy: 0.4733
Epoch 7/150
2250/2250 [==============================] - 2s 807us/step 

2250/2250 [==============================] - 2s 829us/step - loss: 0.8016 - accuracy: 0.5956 - val_loss: 0.8278 - val_accuracy: 0.6080
Epoch 57/150
2250/2250 [==============================] - 2s 805us/step - loss: 0.7967 - accuracy: 0.6009 - val_loss: 0.8512 - val_accuracy: 0.5627
Epoch 58/150
2250/2250 [==============================] - 2s 805us/step - loss: 0.7955 - accuracy: 0.5942 - val_loss: 0.8415 - val_accuracy: 0.5573
Epoch 59/150
2250/2250 [==============================] - 2s 811us/step - loss: 0.7959 - accuracy: 0.5893 - val_loss: 0.8236 - val_accuracy: 0.6067
Epoch 60/150
2250/2250 [==============================] - 2s 818us/step - loss: 0.7923 - accuracy: 0.6058 - val_loss: 0.8447 - val_accuracy: 0.5813
Epoch 61/150
2250/2250 [==============================] - 2s 839us/step - loss: 0.7942 - accuracy: 0.6004 - val_loss: 0.9285 - val_accuracy: 0.5253
Epoch 62/150
2250/2250 [==============================] - 2s 814us/step - loss: 0.7935 - accuracy: 0.5907 - val_loss: 0.8359 

2250/2250 [==============================] - 2s 801us/step - loss: 0.7325 - accuracy: 0.6560 - val_loss: 0.8339 - val_accuracy: 0.5973
Epoch 112/150
2250/2250 [==============================] - 2s 818us/step - loss: 0.7318 - accuracy: 0.6427 - val_loss: 0.8353 - val_accuracy: 0.6040
Epoch 113/150
2250/2250 [==============================] - 2s 812us/step - loss: 0.7273 - accuracy: 0.6484 - val_loss: 0.8458 - val_accuracy: 0.5573
Epoch 114/150
2250/2250 [==============================] - 2s 818us/step - loss: 0.7238 - accuracy: 0.6524 - val_loss: 0.8468 - val_accuracy: 0.5973
Epoch 115/150
2250/2250 [==============================] - 2s 821us/step - loss: 0.7264 - accuracy: 0.6498 - val_loss: 0.9982 - val_accuracy: 0.5133
Epoch 116/150
2250/2250 [==============================] - 2s 841us/step - loss: 0.7307 - accuracy: 0.6458 - val_loss: 0.8331 - val_accuracy: 0.5853
Epoch 117/150
2250/2250 [==============================] - 2s 831us/step - loss: 0.7249 - accuracy: 0.6542 - val_loss: 0

In [12]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

        cats       0.59      0.31      0.41       236
        dogs       0.43      0.70      0.53       236
       panda       0.80      0.67      0.73       278

    accuracy                           0.57       750
   macro avg       0.60      0.56      0.56       750
weighted avg       0.61      0.57      0.57       750



In [13]:
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_accuracy")
plt.plot(N, H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("output/plot.png")

In [14]:
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save("saved_models/simple_nn.model")
f = open("output/simple_nn_lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...
